In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
import pyNBS as nbs

# Preprocess data

In [3]:
fn = '/cellar/data/users/wzhang1984/Firehose/Firehose__2016_01_28/analyses/LUAD/Mutation_Assessor/LUAD-TP.maf.annotated'
df = pd.read_table(fn,low_memory=False)

In [4]:
df = df.loc[(df.loc[:,'is_flank']==0) & (df.loc[:,'is_silent']==0),:]

In [5]:
df['pat'] = df.loc[:,'patient'].str[:12]

In [6]:
df.loc[:,['pat','Hugo_Symbol']].to_csv('../data/LUAD_pat2mut.txt',sep='\t',header=False,index=False)

In [7]:
os.system('cp /cellar/data/users/wzhang1984/forNBS/FIsInGene_031516_with_annotations.txt ../data')
os.system('cp /cellar/data/users/wzhang1984/PCAWG/pat2clin4surv.txt ../data')

0

# Network propagation (iteration)

In [8]:
file_name = '../data/FIsInGene_031516_with_annotations.txt'
output_dir = '../data/'
network, gene2index=nbs.load_network(file_name,output_dir)

* Loading PPI...
	- Edges: 228919
	- Nodes: 12175
* Removing self-loops, multi-edges, and restricting to largest connected component...
	- Largest CC Edges: 228827
	- Largest CC Nodes: 12033
* Saving updated node list to file...


In [9]:
file_name = '../data/LUAD_pat2mut.txt'
mutation_profile, pat2index = nbs.load_mutation(file_name,output_dir,gene2index)

	- Genes in adjacency matrix: 10212
* Saving patient list to file...


In [10]:
rst_prob = 0.4
converge_rate = 0.0001

# run network propagation
pat_diff = nbs.run_diffusion(network,rst_prob,mutation_profile,converge_rate)

# write propagated network on hard disk
with open('{}/prop_pat_mut.npy'.format(output_dir),'w') as file_handle:
    np.save(file_handle,pat_diff)

print 'Finish propagating the data...'

0 iteration: delta is 2.11511608854
1 iteration: delta is 0.539683154566
2 iteration: delta is 0.117364393966
3 iteration: delta is 0.0552451507565
4 iteration: delta is 0.0229251368236
5 iteration: delta is 0.0118859904628
6 iteration: delta is 0.00555218448927
7 iteration: delta is 0.0029636450549
8 iteration: delta is 0.00143451896724
9 iteration: delta is 0.00077921950409
10 iteration: delta is 0.000385843849283
11 iteration: delta is 0.000212511177116
12 iteration: delta is 0.000107291942181
13 iteration: delta is 5.97629771225e-05
Finish propagating the data...


# Netwprk propagation (using PPR matrix)

In [11]:
# Load network

file_name = '../data/FIsInGene_031516_with_annotations.txt'
output_dir = '../data/'
network, gene2index=nbs.load_network(file_name,output_dir)

* Loading PPI...
	- Edges: 228919
	- Nodes: 12175
* Removing self-loops, multi-edges, and restricting to largest connected component...
	- Largest CC Edges: 228827
	- Largest CC Nodes: 12033
* Saving updated node list to file...


In [12]:
# It takes a long time to compute the inverse matrix. But it only has to be done once.

rst_prob = 0.2
network_output_dir = output_dir

PPR = nbs.create_ppr_matrix(network,rst_prob,network_output_dir)

* Creating PPR  matrix...


In [35]:
# load PPR matrix

output_dir = '../data/'
PPR = np.load('{}/ppr_0.2.npy'.format(network_output_dir))

In [36]:
# Load mutations

output_dir = '../data/'
file_name = '../data/LUAD_pat2mut.txt'
mutation_profile, pat2index = nbs.load_mutation(file_name,output_dir,gene2index)

	- Genes in adjacency matrix: 10212
* Saving patient list to file...


In [37]:
# Network propagation

pat_diff = nbs.run_diffusion_PPR(PPR,mutation_profile)

# write propagated network on hard disk
with open('{}/prop_pat_mut.npy'.format(output_dir),'w') as file_handle:
    np.save(file_handle,pat_diff)

print 'Finish propagating the data...'

Finish propagating the data...


# Clustering

In [38]:
M_prop = np.load('../data/prop_pat_mut.npy')
with open('../data/index_genes') as file_handle:
    genes = [a[1] for a in [line.split() for line in file_handle.read().splitlines()]]
with open('../data/index_patients') as file_handle:
    pats = [a[1] for a in [line.split() for line in file_handle.read().splitlines()]]
M_prop=pd.DataFrame(data=M_prop,index=pats,columns=genes)

In [39]:
M_prop_pca, pca_components, explained_variance_ratio = nbs.run_pca(M_prop)

In [40]:
explained_variance_ratio[:100].sum()

0.52207555616169732

In [41]:
labels = nbs.run_clustering_mp(M_prop_pca.iloc[:,:100], 10, nbs.run_SpectralClustering)

Silhouette Score with n_clusters= 2 score: -0.0301750543115
Silhouette Score with n_clusters= 3 score: -0.144581363867
Silhouette Score with n_clusters= 4 score: -0.212887530219
Silhouette Score with n_clusters= 7 score: -0.249667345037
Silhouette Score with n_clusters= 8 score: -0.254130936064
Silhouette Score with n_clusters= 5 score: -0.232685389429
Silhouette Score with n_clusters= 6 score: -0.239299425137
Silhouette Score with n_clusters= 9 score: -0.259830835002
Silhouette Score with n_clusters= 10 score: -0.255337004661


In [42]:
labels.K10.value_counts()

2    62
1    61
0    61
7    57
6    57
4    54
8    52
5    45
3    43
9    41
Name: K10, dtype: int64

# Survival analysis

In [43]:
nbs.run_coxph('../data/pat2clin4surv.txt', labels, '../data/survival/')

0

# Subnetworks

In [44]:
M = pd.DataFrame(data=mutation_profile,index=pats,columns=genes)
K = labels.loc[:,'K8']
test_n_processes = 24
pat2mut_fn = '../data/LUAD_pat2mut.txt'
network_fn = '../data/FIsInGene_031516_with_annotations.txt'
output_dir = '../data/network'
ttest_fdr_cut = 0.3

nbs.subnetwork_wrapper(M, M_prop, K, test_n_processes, pat2mut_fn, network_fn, output_dir, ttest_fdr_cut)

Finish ttest
Finish Fisher exact test
Summarizing subtype signatures
Summarizing subnetworks


0